In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

import pickle 
#import mglearn
import time


from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [56]:
train_df2 = pd.read_csv("wiki_movie_plots_deduped.csv", delimiter=',')
train_df2.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [57]:
train_df = train_df2.iloc[10000:12500]

In [58]:
train_df['Genre length'] = train_df['Genre'].apply(len)
train_df.head()

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length
10000,1982,Partners,American,James Burrows,"John Hurt, Ryan O'Neal",comedy,https://en.wikipedia.org/wiki/Partners_(1982_f...,After a series of murders in Los Angeles's gay...,6
10001,1982,Personal Best,American,Robert Towne,"Mariel Hemingway, Scott Glenn, Patrice Donnelly","drama, sports",https://en.wikipedia.org/wiki/Personal_Best_(f...,Chris Cahill is a young athlete who competes u...,13
10002,1982,Poltergeist,American,Tobe Hooper,"Craig T. Nelson, Beatrice Straight, Dominique ...",horror,https://en.wikipedia.org/wiki/Poltergeist_(198...,Steven and Diane Freeling live a quiet life in...,6
10003,1982,Porky's,American,Bob Clark,"Kim Cattrall, Mark Herrier, Wyatt Knight",comedy,https://en.wikipedia.org/wiki/Porky%27s,A group of Florida high school students plan o...,6
10004,1982,Q,American,Larry Cohen,"Michael Moriarty, Candy Clark, Richard Roundtree",sci-fi,https://en.wikipedia.org/wiki/Q_(film),"The Aztec god Quetzalcoatl, a winged, dragon-l...",6


In [59]:
train_df['Genre length'].describe()

count    2500.000000
mean        7.508000
std         3.305617
min         3.000000
25%         6.000000
50%         6.000000
75%         8.000000
max        26.000000
Name: Genre length, dtype: float64

In [60]:
train_df.describe()

,Release Year,Genre length
count,2500.000000,2500.000000
mean,1989.127600,7.508000
std,3.591946,3.305617
min,1982.000000,3.000000
25%,1986.000000,6.000000
50%,1989.000000,6.000000
75%,1992.000000,8.000000
max,1995.000000,26.000000


In [61]:
train_df['Genre'].nunique()

141

There are 2265 type of genres available in the given dataset

In [62]:
genre_arr = train_df['Genre'].unique()

In [63]:
genre_data = train_df['Genre'].value_counts()

In [64]:
genre_data

comedy              624
drama               544
horror              190
action              169
thriller            105
                   ... 
action, fantasy       1
avant-garde           1
comedy, crime         1
fantasy, romance      1
comedy, mystery       1
Name: Genre, Length: 141, dtype: int64

In [65]:
type(genre_data)

pandas.core.series.Series

In [66]:
import string
alpha = string.ascii_lowercase
alpha_up = string.ascii_uppercase
print(alpha)
print(alpha_up)

abcdefghijklmnopqrstuvwxyz
ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [67]:

def remove_punc(data):
    string = ''
    for i in data:
        if i in alpha:
            string = string + i
        elif i in alpha_up:
            string = string + i
        elif i == ' ':
            string = string + i
        else:
            continue
    return string

In [68]:
dat = 'In a mansion called Xanadu, part of a vast palatial estate in Florida, the elderly Charles Foster Kane is on his deathbed. Holding a snow globe, he utters a word, "Rosebud", and dies; the globe slips from his hand and smashes on the floor'
print(remove_punc(dat))

In a mansion called Xanadu part of a vast palatial estate in Florida the elderly Charles Foster Kane is on his deathbed Holding a snow globe he utters a word Rosebud and dies the globe slips from his hand and smashes on the floor


In [69]:
def genre_corr(data):
    data = data.split()
    final_gen = ''
    list_rom = ['romance','love','love story','musical b', 'romantic','rom-coms','music','musical','actionlove','romanceaction',
               'romancecomedy','romancehorror','romcom','rom\|com','rom',' \(artistic\)',"drama|romance|adult|children"
               ]
    list_act = ['act','action','adventures','kung fu','martial arts','world war ii','world war i','spy film','biker film',
               'buddy cop','buddy film','bruceploitation','drama about child soldiers',"war-time","wartime","ww1","wwii",
               'true crime','crime','\|007','gun fu','afghan war drama','actionadventure','actioncomedy','actiondrama',
                'actionlove','actionmasala','actionchildren','adventurecomedy','actionthriller','martialarts',' \(volleyball\)',
                ' \(aquatics|swimming\)',' \(aquatics|swimming\)',' \(shogi|chess\)',' (road bicycle racing)','american football',
                'dev\|nusrat jahan',' \(road bicycle racing\)','liveaction','heistcomedy','heist','historydisaster','warcomedy',
                'samurai','martial_arts','adventure','spy','superhero',"drama|romance|adult|children",'actionner',
               ]
    list_sus = ['ttriller','coming of age','coming-of-age','slice of life','psycho thriller,',"ero",'actionadventure','dramathriller',
                'dramathriller','thriler','crimethriller','actionthriller','comedysocial','erotica','erotic','comedythriller',
                  'colour\|yellow\|productions\|eros\|international',       'melodrama', 'gangsterthriller',  'ancientcostume', 
                'dramatic','biodrama','bio-drama','comedy-drama adaptation of the mordecai richler novel','drama about child soldiers',
                'drama loosely','slice of life',"comedy–drama"'actionlove','actiondrama','fantasycomedy','dramacomedy',
                'dramacomedysocial','dramathriller','comedydrama','comedyhorror','adventurecomedy','animationdrama','comedysocial',
                'erotica','erotic','biblical','biblical','colour\|yellow\|productions\|eros\|international','liveaction','superheroes',
                'heistcomedy','heist','warcomedy','dramatic','familya','familya','dramedy','dramaa','famil\|','superheroe',
                'devotionalbiography','familydrama','espionage','romancefiction','horrorthriller','suspensethriller','triller',
                'satirical','homosexual','sexual','mockumentary','periodic','politics','tv_miniseries','serial',"musical–comedy",
                "roman|porno","action—masala","horror–thriller",'family','martial_arts','horror','war','adventure','noir',
                'superhero','social','suspense',"drama|romance|adult|children",'actionner',
                ]
    list_sci = ['animated','anime','children\'s','3-d','3d','sci-fi','sci fi','science fiction','avant-garde','animationchildren',
               'computer animation',   ' in animation',   'actionchildren',  'fantasychildren\|','fantasycomedy','fantasyperiod',
                'sciencefiction','animationdrama','fantay','\|\(children\|poker\|karuta\)','superheroes','computeranimation',
                '\|\(fiction\)','science_fictionchildren','science_fiction','superhero',"drama|romance|adult|children"
                ]
    list_hor = ['psychological','j-horror','psycho thriller,',"comedy–horror",'actionadventure','comedyhorror','horror',
                ]
    for i in data:
        if i.lower() in list_rom:
            final_gen = final_gen + ' Romance'
        elif i.lower() in list_act:
            final_gen = final_gen + ' Action'
        elif i.lower() in list_sus:
            final_gen = final_gen + ' Suspense'
        elif i.lower() in list_sci:    
            final_gen = final_gen + ' Science Fiction'
        elif i.lower() in list_hor:
            final_gen = final_gen + ' Horror'
        else:
            final_gen = final_gen + ' Others'
    final_gen = set(final_gen.split())
    genre = ''
    for i in final_gen:
        genre = genre + ' {}'.format(i)
    return genre
            

In [70]:
train_df['Genre String'] = train_df['Genre'].apply(remove_punc)

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String
10000,1982,Partners,American,James Burrows,"John Hurt, Ryan O'Neal",comedy,https://en.wikipedia.org/wiki/Partners_(1982_f...,After a series of murders in Los Angeles's gay...,6,comedy
10001,1982,Personal Best,American,Robert Towne,"Mariel Hemingway, Scott Glenn, Patrice Donnelly","drama, sports",https://en.wikipedia.org/wiki/Personal_Best_(f...,Chris Cahill is a young athlete who competes u...,13,drama sports
10002,1982,Poltergeist,American,Tobe Hooper,"Craig T. Nelson, Beatrice Straight, Dominique ...",horror,https://en.wikipedia.org/wiki/Poltergeist_(198...,Steven and Diane Freeling live a quiet life in...,6,horror
10003,1982,Porky's,American,Bob Clark,"Kim Cattrall, Mark Herrier, Wyatt Knight",comedy,https://en.wikipedia.org/wiki/Porky%27s,A group of Florida high school students plan o...,6,comedy
10004,1982,Q,American,Larry Cohen,"Michael Moriarty, Candy Clark, Richard Roundtree",sci-fi,https://en.wikipedia.org/wiki/Q_(film),"The Aztec god Quetzalcoatl, a winged, dragon-l...",6,scifi
10005,1982,Rocky III,American,Sylvester Stallone,"Sylvester Stallone, Carl Weathers, Talia Shire...",drama,https://en.wikipedia.org/wiki/Rocky_III,Three years after winning the heavyweight cham...,5,drama
10006,1982,Savannah Smiles,American,Pierre de Moro,"Mark Miller, Donovan Scott",comedy,https://en.wikipedia.org/wiki/Savannah_Smiles,The story begins at a prison work site where B...,6,comedy
10007,1982,The Secret of NIMH,American,Don Bluth,"Voices of Dom DeLuise, Peter Strauss",animated,https://en.wikipedia.org/wiki/The_Secret_of_NIMH,"Mrs. Brisby, a timid widowed field mouse, live...",8,animated
10008,1982,The Seduction,American,David Schmoeller,"Morgan Fairchild, Andrew Stevens, Colleen Camp",thriller,https://en.wikipedia.org/wiki/The_Seduction_(f...,Jamie Douglas is a beautiful TV newswoman with...,8,thriller
10009,1982,The Sender,American,Roger Christian,"Kathryn Harrold, Paul Freeman",horror,https://en.wikipedia.org/wiki/The_Sender,A young man is admitted to a state mental hosp...,6,horror


In [72]:
train_df['Genre Corrected'] = train_df['Genre String'].apply(genre_corr)

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected
10000,1982,Partners,American,James Burrows,"John Hurt, Ryan O'Neal",comedy,https://en.wikipedia.org/wiki/Partners_(1982_f...,After a series of murders in Los Angeles's gay...,6,comedy,Others
10001,1982,Personal Best,American,Robert Towne,"Mariel Hemingway, Scott Glenn, Patrice Donnelly","drama, sports",https://en.wikipedia.org/wiki/Personal_Best_(f...,Chris Cahill is a young athlete who competes u...,13,drama sports,Others
10002,1982,Poltergeist,American,Tobe Hooper,"Craig T. Nelson, Beatrice Straight, Dominique ...",horror,https://en.wikipedia.org/wiki/Poltergeist_(198...,Steven and Diane Freeling live a quiet life in...,6,horror,Suspense
10003,1982,Porky's,American,Bob Clark,"Kim Cattrall, Mark Herrier, Wyatt Knight",comedy,https://en.wikipedia.org/wiki/Porky%27s,A group of Florida high school students plan o...,6,comedy,Others
10004,1982,Q,American,Larry Cohen,"Michael Moriarty, Candy Clark, Richard Roundtree",sci-fi,https://en.wikipedia.org/wiki/Q_(film),"The Aztec god Quetzalcoatl, a winged, dragon-l...",6,scifi,Others
10005,1982,Rocky III,American,Sylvester Stallone,"Sylvester Stallone, Carl Weathers, Talia Shire...",drama,https://en.wikipedia.org/wiki/Rocky_III,Three years after winning the heavyweight cham...,5,drama,Others
10006,1982,Savannah Smiles,American,Pierre de Moro,"Mark Miller, Donovan Scott",comedy,https://en.wikipedia.org/wiki/Savannah_Smiles,The story begins at a prison work site where B...,6,comedy,Others
10007,1982,The Secret of NIMH,American,Don Bluth,"Voices of Dom DeLuise, Peter Strauss",animated,https://en.wikipedia.org/wiki/The_Secret_of_NIMH,"Mrs. Brisby, a timid widowed field mouse, live...",8,animated,Science Fiction
10008,1982,The Seduction,American,David Schmoeller,"Morgan Fairchild, Andrew Stevens, Colleen Camp",thriller,https://en.wikipedia.org/wiki/The_Seduction_(f...,Jamie Douglas is a beautiful TV newswoman with...,8,thriller,Others
10009,1982,The Sender,American,Roger Christian,"Kathryn Harrold, Paul Freeman",horror,https://en.wikipedia.org/wiki/The_Sender,A young man is admitted to a state mental hosp...,6,horror,Suspense


In [74]:
train_df['Genre Corrected 2'] = train_df['Genre'].apply(genre_corr)

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected,Genre Corrected 2
10000,1982,Partners,American,James Burrows,"John Hurt, Ryan O'Neal",comedy,https://en.wikipedia.org/wiki/Partners_(1982_f...,After a series of murders in Los Angeles's gay...,6,comedy,Others,Others
10001,1982,Personal Best,American,Robert Towne,"Mariel Hemingway, Scott Glenn, Patrice Donnelly","drama, sports",https://en.wikipedia.org/wiki/Personal_Best_(f...,Chris Cahill is a young athlete who competes u...,13,drama sports,Others,Others
10002,1982,Poltergeist,American,Tobe Hooper,"Craig T. Nelson, Beatrice Straight, Dominique ...",horror,https://en.wikipedia.org/wiki/Poltergeist_(198...,Steven and Diane Freeling live a quiet life in...,6,horror,Suspense,Suspense
10003,1982,Porky's,American,Bob Clark,"Kim Cattrall, Mark Herrier, Wyatt Knight",comedy,https://en.wikipedia.org/wiki/Porky%27s,A group of Florida high school students plan o...,6,comedy,Others,Others
10004,1982,Q,American,Larry Cohen,"Michael Moriarty, Candy Clark, Richard Roundtree",sci-fi,https://en.wikipedia.org/wiki/Q_(film),"The Aztec god Quetzalcoatl, a winged, dragon-l...",6,scifi,Others,Science Fiction
10005,1982,Rocky III,American,Sylvester Stallone,"Sylvester Stallone, Carl Weathers, Talia Shire...",drama,https://en.wikipedia.org/wiki/Rocky_III,Three years after winning the heavyweight cham...,5,drama,Others,Others
10006,1982,Savannah Smiles,American,Pierre de Moro,"Mark Miller, Donovan Scott",comedy,https://en.wikipedia.org/wiki/Savannah_Smiles,The story begins at a prison work site where B...,6,comedy,Others,Others
10007,1982,The Secret of NIMH,American,Don Bluth,"Voices of Dom DeLuise, Peter Strauss",animated,https://en.wikipedia.org/wiki/The_Secret_of_NIMH,"Mrs. Brisby, a timid widowed field mouse, live...",8,animated,Science Fiction,Science Fiction
10008,1982,The Seduction,American,David Schmoeller,"Morgan Fairchild, Andrew Stevens, Colleen Camp",thriller,https://en.wikipedia.org/wiki/The_Seduction_(f...,Jamie Douglas is a beautiful TV newswoman with...,8,thriller,Others,Others
10009,1982,The Sender,American,Roger Christian,"Kathryn Harrold, Paul Freeman",horror,https://en.wikipedia.org/wiki/The_Sender,A young man is admitted to a state mental hosp...,6,horror,Suspense,Suspense


In [76]:
train_df['Genre Corrected'].value_counts()

 Others                     1690
 Suspense                    262
 Action                      222
 Action Others               131
 Others Suspense              52
 Science Fiction              48
 Others Romance               44
 Romance                      29
 Others Science Fiction       10
 Action Suspense               9
 Action Others Suspense        1
 Science Romance Fiction       1
 Action Romance                1
Name: Genre Corrected, dtype: int64

In [77]:
train_df['Genre Corrected 2'].value_counts()

 Others                    1659
 Suspense                   262
 Action                     221
 Action Others              121
 Science Fiction            104
 Others Romance              45
 Others Suspense             44
 Romance                     28
 Others Science Fiction      13
 Action Others Suspense       1
 Action Science Fiction       1
 Action Suspense              1
Name: Genre Corrected 2, dtype: int64

In [78]:
train_df.drop(['Genre Corrected 2'],axis=1,inplace=True)

c:\Miniconda\envs\py37_default\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [79]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected
10000,1982,Partners,American,James Burrows,"John Hurt, Ryan O'Neal",comedy,https://en.wikipedia.org/wiki/Partners_(1982_f...,After a series of murders in Los Angeles's gay...,6,comedy,Others
10001,1982,Personal Best,American,Robert Towne,"Mariel Hemingway, Scott Glenn, Patrice Donnelly","drama, sports",https://en.wikipedia.org/wiki/Personal_Best_(f...,Chris Cahill is a young athlete who competes u...,13,drama sports,Others
10002,1982,Poltergeist,American,Tobe Hooper,"Craig T. Nelson, Beatrice Straight, Dominique ...",horror,https://en.wikipedia.org/wiki/Poltergeist_(198...,Steven and Diane Freeling live a quiet life in...,6,horror,Suspense
10003,1982,Porky's,American,Bob Clark,"Kim Cattrall, Mark Herrier, Wyatt Knight",comedy,https://en.wikipedia.org/wiki/Porky%27s,A group of Florida high school students plan o...,6,comedy,Others
10004,1982,Q,American,Larry Cohen,"Michael Moriarty, Candy Clark, Richard Roundtree",sci-fi,https://en.wikipedia.org/wiki/Q_(film),"The Aztec god Quetzalcoatl, a winged, dragon-l...",6,scifi,Others
10005,1982,Rocky III,American,Sylvester Stallone,"Sylvester Stallone, Carl Weathers, Talia Shire...",drama,https://en.wikipedia.org/wiki/Rocky_III,Three years after winning the heavyweight cham...,5,drama,Others
10006,1982,Savannah Smiles,American,Pierre de Moro,"Mark Miller, Donovan Scott",comedy,https://en.wikipedia.org/wiki/Savannah_Smiles,The story begins at a prison work site where B...,6,comedy,Others
10007,1982,The Secret of NIMH,American,Don Bluth,"Voices of Dom DeLuise, Peter Strauss",animated,https://en.wikipedia.org/wiki/The_Secret_of_NIMH,"Mrs. Brisby, a timid widowed field mouse, live...",8,animated,Science Fiction
10008,1982,The Seduction,American,David Schmoeller,"Morgan Fairchild, Andrew Stevens, Colleen Camp",thriller,https://en.wikipedia.org/wiki/The_Seduction_(f...,Jamie Douglas is a beautiful TV newswoman with...,8,thriller,Others
10009,1982,The Sender,American,Roger Christian,"Kathryn Harrold, Paul Freeman",horror,https://en.wikipedia.org/wiki/The_Sender,A young man is admitted to a state mental hosp...,6,horror,Suspense


In [80]:
train_df['GenreSplit']=train_df['Genre Corrected'].str.split()

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected,GenreSplit
10000,1982,Partners,American,James Burrows,"John Hurt, Ryan O'Neal",comedy,https://en.wikipedia.org/wiki/Partners_(1982_f...,After a series of murders in Los Angeles's gay...,6,comedy,Others,[Others]
10001,1982,Personal Best,American,Robert Towne,"Mariel Hemingway, Scott Glenn, Patrice Donnelly","drama, sports",https://en.wikipedia.org/wiki/Personal_Best_(f...,Chris Cahill is a young athlete who competes u...,13,drama sports,Others,[Others]
10002,1982,Poltergeist,American,Tobe Hooper,"Craig T. Nelson, Beatrice Straight, Dominique ...",horror,https://en.wikipedia.org/wiki/Poltergeist_(198...,Steven and Diane Freeling live a quiet life in...,6,horror,Suspense,[Suspense]
10003,1982,Porky's,American,Bob Clark,"Kim Cattrall, Mark Herrier, Wyatt Knight",comedy,https://en.wikipedia.org/wiki/Porky%27s,A group of Florida high school students plan o...,6,comedy,Others,[Others]
10004,1982,Q,American,Larry Cohen,"Michael Moriarty, Candy Clark, Richard Roundtree",sci-fi,https://en.wikipedia.org/wiki/Q_(film),"The Aztec god Quetzalcoatl, a winged, dragon-l...",6,scifi,Others,[Others]
10005,1982,Rocky III,American,Sylvester Stallone,"Sylvester Stallone, Carl Weathers, Talia Shire...",drama,https://en.wikipedia.org/wiki/Rocky_III,Three years after winning the heavyweight cham...,5,drama,Others,[Others]
10006,1982,Savannah Smiles,American,Pierre de Moro,"Mark Miller, Donovan Scott",comedy,https://en.wikipedia.org/wiki/Savannah_Smiles,The story begins at a prison work site where B...,6,comedy,Others,[Others]
10007,1982,The Secret of NIMH,American,Don Bluth,"Voices of Dom DeLuise, Peter Strauss",animated,https://en.wikipedia.org/wiki/The_Secret_of_NIMH,"Mrs. Brisby, a timid widowed field mouse, live...",8,animated,Science Fiction,"[Science, Fiction]"
10008,1982,The Seduction,American,David Schmoeller,"Morgan Fairchild, Andrew Stevens, Colleen Camp",thriller,https://en.wikipedia.org/wiki/The_Seduction_(f...,Jamie Douglas is a beautiful TV newswoman with...,8,thriller,Others,[Others]
10009,1982,The Sender,American,Roger Christian,"Kathryn Harrold, Paul Freeman",horror,https://en.wikipedia.org/wiki/The_Sender,A young man is admitted to a state mental hosp...,6,horror,Suspense,[Suspense]


In [82]:
def text_process(word):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    
    # Now just remove any stopwords
    return [i for i in word.split() if i.lower() not in stopwords.words('english')]

In [83]:
t = 10000
for i in range(t,t+5):
    print(train_df['Plot'][i])

After a series of murders in Los Angeles's gay community, heterosexual police officer Sgt. Benson is assigned to go undercover as half of a gay couple with Officer Kerwin, a Records Clerk. Kerwin naively believes that he is closeted, although the entire Department knows about his sexual identity. The pair discover an earlier murder and learn that both victims appeared in the same gay magazine. Each had received a call from a hoarse-voiced man asking them to model for him, only to turn up dead soon after. Benson models for the magazine and is approached by the same hoarse-voiced man; but, when another model turns up dead, the man is cleared as a suspect.
Benson grows close to Jill, the photographer of his shoot, and plans a weekend getaway with her. Kerwin suspects her of the murders, but his superiors put it down to jealousy. Kerwin uncovers evidence implicating Jill; but, when the police move to apprehend her, they discover her corpse. Her death unknown to Benson, he arrives for his r

In [84]:
train_df['Plot without punc'] = train_df['Plot'].apply(remove_punc)

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
train_df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected,GenreSplit,Plot without punc
10000,1982,Partners,American,James Burrows,"John Hurt, Ryan O'Neal",comedy,https://en.wikipedia.org/wiki/Partners_(1982_f...,After a series of murders in Los Angeles's gay...,6,comedy,Others,[Others],After a series of murders in Los Angeless gay ...
10001,1982,Personal Best,American,Robert Towne,"Mariel Hemingway, Scott Glenn, Patrice Donnelly","drama, sports",https://en.wikipedia.org/wiki/Personal_Best_(f...,Chris Cahill is a young athlete who competes u...,13,drama sports,Others,[Others],Chris Cahill is a young athlete who competes u...
10002,1982,Poltergeist,American,Tobe Hooper,"Craig T. Nelson, Beatrice Straight, Dominique ...",horror,https://en.wikipedia.org/wiki/Poltergeist_(198...,Steven and Diane Freeling live a quiet life in...,6,horror,Suspense,[Suspense],Steven and Diane Freeling live a quiet life in...
10003,1982,Porky's,American,Bob Clark,"Kim Cattrall, Mark Herrier, Wyatt Knight",comedy,https://en.wikipedia.org/wiki/Porky%27s,A group of Florida high school students plan o...,6,comedy,Others,[Others],A group of Florida high school students plan o...
10004,1982,Q,American,Larry Cohen,"Michael Moriarty, Candy Clark, Richard Roundtree",sci-fi,https://en.wikipedia.org/wiki/Q_(film),"The Aztec god Quetzalcoatl, a winged, dragon-l...",6,scifi,Others,[Others],The Aztec god Quetzalcoatl a winged dragonlike...


In [86]:
train_df.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'Genre length', 'Genre String',
       'Genre Corrected', 'GenreSplit', 'Plot without punc'],
      dtype='object')

In [87]:
train_df.drop(['Release Year','Origin/Ethnicity', 'Director', 'Cast','Genre', 'Wiki Page','Genre length', 'Genre String',
              'GenreSplit',
              ],axis = 1, inplace = True)

c:\Miniconda\envs\py37_default\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [88]:
train_df.head()

,Title,Plot,Genre Corrected,Plot without punc
10000,Partners,After a series of murders in Los Angeles's gay...,Others,After a series of murders in Los Angeless gay ...
10001,Personal Best,Chris Cahill is a young athlete who competes u...,Others,Chris Cahill is a young athlete who competes u...
10002,Poltergeist,Steven and Diane Freeling live a quiet life in...,Suspense,Steven and Diane Freeling live a quiet life in...
10003,Porky's,A group of Florida high school students plan o...,Others,A group of Florida high school students plan o...
10004,Q,"The Aztec god Quetzalcoatl, a winged, dragon-l...",Others,The Aztec god Quetzalcoatl a winged dragonlike...


In [89]:
train_df.reset_index(drop = True,inplace = True)
train_df.head()

,Title,Plot,Genre Corrected,Plot without punc
0,Partners,After a series of murders in Los Angeles's gay...,Others,After a series of murders in Los Angeless gay ...
1,Personal Best,Chris Cahill is a young athlete who competes u...,Others,Chris Cahill is a young athlete who competes u...
2,Poltergeist,Steven and Diane Freeling live a quiet life in...,Suspense,Steven and Diane Freeling live a quiet life in...
3,Porky's,A group of Florida high school students plan o...,Others,A group of Florida high school students plan o...
4,Q,"The Aztec god Quetzalcoatl, a winged, dragon-l...",Others,The Aztec god Quetzalcoatl a winged dragonlike...


In [90]:
train_df['Plot Length'] = train_df['Plot without punc'].apply(len)

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [91]:
train_df.head()

,Title,Plot,Genre Corrected,Plot without punc,Plot Length
0,Partners,After a series of murders in Los Angeles's gay...,Others,After a series of murders in Los Angeless gay ...,1329
1,Personal Best,Chris Cahill is a young athlete who competes u...,Others,Chris Cahill is a young athlete who competes u...,693
2,Poltergeist,Steven and Diane Freeling live a quiet life in...,Suspense,Steven and Diane Freeling live a quiet life in...,3869
3,Porky's,A group of Florida high school students plan o...,Others,A group of Florida high school students plan o...,2895
4,Q,"The Aztec god Quetzalcoatl, a winged, dragon-l...",Others,The Aztec god Quetzalcoatl a winged dragonlike...,1266


In [92]:
train_df['Plot Length'].sum()

6879158

In [93]:
train_df.head()

,Title,Plot,Genre Corrected,Plot without punc,Plot Length
0,Partners,After a series of murders in Los Angeles's gay...,Others,After a series of murders in Los Angeless gay ...,1329
1,Personal Best,Chris Cahill is a young athlete who competes u...,Others,Chris Cahill is a young athlete who competes u...,693
2,Poltergeist,Steven and Diane Freeling live a quiet life in...,Suspense,Steven and Diane Freeling live a quiet life in...,3869
3,Porky's,A group of Florida high school students plan o...,Others,A group of Florida high school students plan o...,2895
4,Q,"The Aztec god Quetzalcoatl, a winged, dragon-l...",Others,The Aztec god Quetzalcoatl a winged dragonlike...,1266


In [94]:
from nltk.corpus import stopwords

In [95]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\navendu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [96]:
train_df['Plot final clean'] = train_df['Plot without punc'].apply(text_process)

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [97]:
train_df.columns

Index(['Title', 'Plot', 'Genre Corrected', 'Plot without punc', 'Plot Length',
       'Plot final clean'],
      dtype='object')

In [98]:
train_df.head()

,Title,Plot,Genre Corrected,Plot without punc,Plot Length,Plot final clean
0,Partners,After a series of murders in Los Angeles's gay...,Others,After a series of murders in Los Angeless gay ...,1329,"[series, murders, Los, Angeless, gay, communit..."
1,Personal Best,Chris Cahill is a young athlete who competes u...,Others,Chris Cahill is a young athlete who competes u...,693,"[Chris, Cahill, young, athlete, competes, unsu..."
2,Poltergeist,Steven and Diane Freeling live a quiet life in...,Suspense,Steven and Diane Freeling live a quiet life in...,3869,"[Steven, Diane, Freeling, live, quiet, life, O..."
3,Porky's,A group of Florida high school students plan o...,Others,A group of Florida high school students plan o...,2895,"[group, Florida, high, school, students, plan,..."
4,Q,"The Aztec god Quetzalcoatl, a winged, dragon-l...",Others,The Aztec god Quetzalcoatl a winged dragonlike...,1266,"[Aztec, god, Quetzalcoatl, winged, dragonlike,..."


In [99]:
train_df.drop([ 'Plot', 'Plot without punc', 'Plot Length',],axis = 1, inplace = True)

c:\Miniconda\envs\py37_default\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [100]:
train_df.head()

,Title,Genre Corrected,Plot final clean
0,Partners,Others,"[series, murders, Los, Angeless, gay, communit..."
1,Personal Best,Others,"[Chris, Cahill, young, athlete, competes, unsu..."
2,Poltergeist,Suspense,"[Steven, Diane, Freeling, live, quiet, life, O..."
3,Porky's,Others,"[group, Florida, high, school, students, plan,..."
4,Q,Others,"[Aztec, god, Quetzalcoatl, winged, dragonlike,..."


In [54]:
train_df.columns

Index(['Genre Corrected', 'Plot without punc', 'Plot Length'], dtype='object')

In [101]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

In [102]:
def stem(name):
    name_list = []
    ps = PorterStemmer()
    for i in name:
        words = word_tokenize(i)
        name_list.append(words)
    return name_list

In [105]:
train_df['Plot Stem'] = train_df['Plot final clean'].apply(stem)
train_df.head()

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Title,Genre Corrected,Plot final clean,Plot Stem
0,Partners,Others,"[series, murders, Los, Angeless, gay, communit...","[[series], [murders], [Los], [Angeless], [gay]..."
1,Personal Best,Others,"[Chris, Cahill, young, athlete, competes, unsu...","[[Chris], [Cahill], [young], [athlete], [compe..."
2,Poltergeist,Suspense,"[Steven, Diane, Freeling, live, quiet, life, O...","[[Steven], [Diane], [Freeling], [live], [quiet..."
3,Porky's,Others,"[group, Florida, high, school, students, plan,...","[[group], [Florida], [high], [school], [studen..."
4,Q,Others,"[Aztec, god, Quetzalcoatl, winged, dragonlike,...","[[Aztec], [god], [Quetzalcoatl], [winged], [dr..."


In [106]:
def uniq_list(name):
    name_list = []
    for i in name:
        if i in name_list:
            continue
        else:
            name_list.append(i)
    return name_list

In [107]:
train_df['Plot After Stem'] = train_df["Plot Stem"].apply(uniq_list)
train_df.head()

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Title,Genre Corrected,Plot final clean,Plot Stem,Plot After Stem
0,Partners,Others,"[series, murders, Los, Angeless, gay, communit...","[[series], [murders], [Los], [Angeless], [gay]...","[[series], [murders], [Los], [Angeless], [gay]..."
1,Personal Best,Others,"[Chris, Cahill, young, athlete, competes, unsu...","[[Chris], [Cahill], [young], [athlete], [compe...","[[Chris], [Cahill], [young], [athlete], [compe..."
2,Poltergeist,Suspense,"[Steven, Diane, Freeling, live, quiet, life, O...","[[Steven], [Diane], [Freeling], [live], [quiet...","[[Steven], [Diane], [Freeling], [live], [quiet..."
3,Porky's,Others,"[group, Florida, high, school, students, plan,...","[[group], [Florida], [high], [school], [studen...","[[group], [Florida], [high], [school], [studen..."
4,Q,Others,"[Aztec, god, Quetzalcoatl, winged, dragonlike,...","[[Aztec], [god], [Quetzalcoatl], [winged], [dr...","[[Aztec], [god], [Quetzalcoatl], [winged], [dr..."


In [108]:
train_df["Plot final clean Length"] = train_df['Plot final clean'].apply(len)
train_df["Plot Stem Length"] = train_df['Plot Stem'].apply(len)
train_df["Plot After Stem Length"] = train_df['Plot After Stem'].apply(len)
train_df.head()

c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Miniconda\envs\py37_default\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,Title,Genre Corrected,Plot final clean,Plot Stem,Plot After Stem,Plot final clean Length,Plot Stem Length,Plot After Stem Length
0,Partners,Others,"[series, murders, Los, Angeless, gay, communit...","[[series], [murders], [Los], [Angeless], [gay]...","[[series], [murders], [Los], [Angeless], [gay]...",126,126,93
1,Personal Best,Others,"[Chris, Cahill, young, athlete, competes, unsu...","[[Chris], [Cahill], [young], [athlete], [compe...","[[Chris], [Cahill], [young], [athlete], [compe...",69,69,60
2,Poltergeist,Suspense,"[Steven, Diane, Freeling, live, quiet, life, O...","[[Steven], [Diane], [Freeling], [live], [quiet...","[[Steven], [Diane], [Freeling], [live], [quiet...",360,360,250
3,Porky's,Others,"[group, Florida, high, school, students, plan,...","[[group], [Florida], [high], [school], [studen...","[[group], [Florida], [high], [school], [studen...",271,271,204
4,Q,Others,"[Aztec, god, Quetzalcoatl, winged, dragonlike,...","[[Aztec], [god], [Quetzalcoatl], [winged], [dr...","[[Aztec], [god], [Quetzalcoatl], [winged], [dr...",132,132,121


In [109]:
train_df.columns

Index(['Title', 'Genre Corrected', 'Plot final clean', 'Plot Stem',
       'Plot After Stem', 'Plot final clean Length', 'Plot Stem Length',
       'Plot After Stem Length'],
      dtype='object')

In [110]:
train_df.drop([ 'Plot final clean', 'Plot Stem', 'Plot final clean Length', 'Plot Stem Length',
       'Plot After Stem Length'],axis = 1, inplace =True)
train_df.head()

c:\Miniconda\envs\py37_default\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Title,Genre Corrected,Plot After Stem
0,Partners,Others,"[[series], [murders], [Los], [Angeless], [gay]..."
1,Personal Best,Others,"[[Chris], [Cahill], [young], [athlete], [compe..."
2,Poltergeist,Suspense,"[[Steven], [Diane], [Freeling], [live], [quiet..."
3,Porky's,Others,"[[group], [Florida], [high], [school], [studen..."
4,Q,Others,"[[Aztec], [god], [Quetzalcoatl], [winged], [dr..."


In [111]:
test_df = pd.read_csv('test_data.csv')

In [112]:
test_df.head()

,Unnamed: 0,Movie_Title,Release_date,Movie_cast,Movie_plot,plot_length
0,0,Citizen Kane,"May 1, 1941Palace Theatre) September 5, 1941",Orson Welles Joseph Cotten Dorothy Comingore E...,"In a mansion called Xanadu, part of a vast pal...",3751
1,1,The Rules of the Game,7 July 1939 (Paris),Nora GregorPaulette DubostMarcel DalioRoland T...,Aviator André Jurieux (Roland Toutain) lands a...,3056
2,2,Bicycle Thieves,24 November 1948,Enzo Staiola Lamberto Maggiorani,In the post-World War II Val Melaina neighbour...,2477
3,3,Vertigo (film),"May 9, 1958",James Stewart Kim Novak Barbara Bel Geddes Tom...,"After a rooftop chase, where a fellow policema...",3947
4,4,The Godfather,"March 14, 1972Loew's State Theatre) March 24, ...",Marlon Brando Al Pacino James Caan Richard Cas...,"In 1945 New York City, at his daughter Connie'...",4114


In [114]:
test_df.columns

Index(['Unnamed: 0', 'Movie_Title', 'Release_date', 'Movie_cast', 'Movie_plot',
       'plot_length'],
      dtype='object')

In [115]:
test_df.drop(['Unnamed: 0', 'Release_date', 'Movie_cast'],axis = 1, inplace = True)
test_df.head()

,Movie_Title,Movie_plot,plot_length
0,Citizen Kane,"In a mansion called Xanadu, part of a vast pal...",3751
1,The Rules of the Game,Aviator André Jurieux (Roland Toutain) lands a...,3056
2,Bicycle Thieves,In the post-World War II Val Melaina neighbour...,2477
3,Vertigo (film),"After a rooftop chase, where a fellow policema...",3947
4,The Godfather,"In 1945 New York City, at his daughter Connie'...",4114


In [ ]:
def remove_alpha(mess):
    

In [121]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    mess = str(mess)
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation and type(char)== str]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [124]:
test_df['Plot without punc and stopwords'] = test_df['Movie_plot'].apply(text_process)

In [125]:
test_df.head()

,Movie_Title,Movie_plot,plot_length,Plot without punc,Plot without punc and stopwords
0,Citizen Kane,"In a mansion called Xanadu, part of a vast pal...",3751,"[mansion, called, Xanadu, part, vast, palatial...","[mansion, called, Xanadu, part, vast, palatial..."
1,The Rules of the Game,Aviator André Jurieux (Roland Toutain) lands a...,3056,"[Aviator, André, Jurieux, Roland, Toutain, lan...","[Aviator, André, Jurieux, Roland, Toutain, lan..."
2,Bicycle Thieves,In the post-World War II Val Melaina neighbour...,2477,"[postWorld, War, II, Val, Melaina, neighbourho...","[postWorld, War, II, Val, Melaina, neighbourho..."
3,Vertigo (film),"After a rooftop chase, where a fellow policema...",3947,"[rooftop, chase, fellow, policeman, falls, dea...","[rooftop, chase, fellow, policeman, falls, dea..."
4,The Godfather,"In 1945 New York City, at his daughter Connie'...",4114,"[1945, New, York, City, daughter, Connies, wed...","[1945, New, York, City, daughter, Connies, wed..."


In [126]:
train_df.to_csv('train_data_csv')

In [128]:
test_df['Plot without punc length'] = test_df['Plot without punc'].apply(len)
test_df['Plot without punc and stopwords len'] = test_df['Plot without punc and stopwords'].apply(len)
test_df.head()

,Movie_Title,Movie_plot,plot_length,Plot without punc,Plot without punc and stopwords,Plot without punc length,Plot without punc and stopwords len
0,Citizen Kane,"In a mansion called Xanadu, part of a vast pal...",3751,"[mansion, called, Xanadu, part, vast, palatial...","[mansion, called, Xanadu, part, vast, palatial...",365,365
1,The Rules of the Game,Aviator André Jurieux (Roland Toutain) lands a...,3056,"[Aviator, André, Jurieux, Roland, Toutain, lan...","[Aviator, André, Jurieux, Roland, Toutain, lan...",292,292
2,Bicycle Thieves,In the post-World War II Val Melaina neighbour...,2477,"[postWorld, War, II, Val, Melaina, neighbourho...","[postWorld, War, II, Val, Melaina, neighbourho...",244,244
3,Vertigo (film),"After a rooftop chase, where a fellow policema...",3947,"[rooftop, chase, fellow, policeman, falls, dea...","[rooftop, chase, fellow, policeman, falls, dea...",363,363
4,The Godfather,"In 1945 New York City, at his daughter Connie'...",4114,"[1945, New, York, City, daughter, Connies, wed...","[1945, New, York, City, daughter, Connies, wed...",406,406


In [129]:
test_df.columns

Index(['Movie_Title', 'Movie_plot', 'plot_length', 'Plot without punc',
       'Plot without punc and stopwords', 'Plot without punc length',
       'Plot without punc and stopwords len'],
      dtype='object')

In [130]:
test_df.drop(['Movie_plot', 'plot_length', 'Plot without punc',
       'Plot without punc length',
       'Plot without punc and stopwords len'],axis = 1, inplace = True)
test_df.head()

,Movie_Title,Plot without punc and stopwords
0,Citizen Kane,"[mansion, called, Xanadu, part, vast, palatial..."
1,The Rules of the Game,"[Aviator, André, Jurieux, Roland, Toutain, lan..."
2,Bicycle Thieves,"[postWorld, War, II, Val, Melaina, neighbourho..."
3,Vertigo (film),"[rooftop, chase, fellow, policeman, falls, dea..."
4,The Godfather,"[1945, New, York, City, daughter, Connies, wed..."


In [132]:
test_df['Plot Stem'] = test_df['Plot without punc and stopwords'].apply(stem)
test_df.head()

,Movie_Title,Plot without punc and stopwords,Plot Stem
0,Citizen Kane,"[mansion, called, Xanadu, part, vast, palatial...","[[mansion], [called], [Xanadu], [part], [vast]..."
1,The Rules of the Game,"[Aviator, André, Jurieux, Roland, Toutain, lan...","[[Aviator], [André], [Jurieux], [Roland], [Tou..."
2,Bicycle Thieves,"[postWorld, War, II, Val, Melaina, neighbourho...","[[postWorld], [War], [II], [Val], [Melaina], [..."
3,Vertigo (film),"[rooftop, chase, fellow, policeman, falls, dea...","[[rooftop], [chase], [fellow], [policeman], [f..."
4,The Godfather,"[1945, New, York, City, daughter, Connies, wed...","[[1945], [New], [York], [City], [daughter], [C..."


In [133]:
test_df['Plot After Stem'] = test_df["Plot Stem"].apply(uniq_list)
test_df.head()

,Movie_Title,Plot without punc and stopwords,Plot Stem,Plot After Stem
0,Citizen Kane,"[mansion, called, Xanadu, part, vast, palatial...","[[mansion], [called], [Xanadu], [part], [vast]...","[[mansion], [called], [Xanadu], [part], [vast]..."
1,The Rules of the Game,"[Aviator, André, Jurieux, Roland, Toutain, lan...","[[Aviator], [André], [Jurieux], [Roland], [Tou...","[[Aviator], [André], [Jurieux], [Roland], [Tou..."
2,Bicycle Thieves,"[postWorld, War, II, Val, Melaina, neighbourho...","[[postWorld], [War], [II], [Val], [Melaina], [...","[[postWorld], [War], [II], [Val], [Melaina], [..."
3,Vertigo (film),"[rooftop, chase, fellow, policeman, falls, dea...","[[rooftop], [chase], [fellow], [policeman], [f...","[[rooftop], [chase], [fellow], [policeman], [f..."
4,The Godfather,"[1945, New, York, City, daughter, Connies, wed...","[[1945], [New], [York], [City], [daughter], [C...","[[1945], [New], [York], [City], [daughter], [C..."


In [135]:
test_df["Plot without punc and stopwords len"] = test_df['Plot without punc and stopwords'].apply(len)
test_df["Plot Stem len"] = test_df['Plot Stem'].apply(len)
test_df["Plot After Stem len"] = test_df['Plot After Stem'].apply(len)
test_df.head()

,Movie_Title,Plot without punc and stopwords,Plot Stem,Plot After Stem,Plot without punc and stopwords len,Plot Stem len,Plot After Stem len
0,Citizen Kane,"[mansion, called, Xanadu, part, vast, palatial...","[[mansion], [called], [Xanadu], [part], [vast]...","[[mansion], [called], [Xanadu], [part], [vast]...",365,365,266
1,The Rules of the Game,"[Aviator, André, Jurieux, Roland, Toutain, lan...","[[Aviator], [André], [Jurieux], [Roland], [Tou...","[[Aviator], [André], [Jurieux], [Roland], [Tou...",292,292,195
2,Bicycle Thieves,"[postWorld, War, II, Val, Melaina, neighbourho...","[[postWorld], [War], [II], [Val], [Melaina], [...","[[postWorld], [War], [II], [Val], [Melaina], [...",244,244,186
3,Vertigo (film),"[rooftop, chase, fellow, policeman, falls, dea...","[[rooftop], [chase], [fellow], [policeman], [f...","[[rooftop], [chase], [fellow], [policeman], [f...",363,363,249
4,The Godfather,"[1945, New, York, City, daughter, Connies, wed...","[[1945], [New], [York], [City], [daughter], [C...","[[1945], [New], [York], [City], [daughter], [C...",406,406,290


In [136]:
test_df.columns

Index(['Movie_Title', 'Plot without punc and stopwords', 'Plot Stem',
       'Plot After Stem', 'Plot without punc and stopwords len',
       'Plot Stem len', 'Plot After Stem len'],
      dtype='object')

In [137]:
test_df.drop(['Plot without punc and stopwords', 'Plot Stem',
       'Plot without punc and stopwords len',
       'Plot Stem len', 'Plot After Stem len'],axis=1,inplace=True)

In [138]:
test_df.head()

,Movie_Title,Plot After Stem
0,Citizen Kane,"[[mansion], [called], [Xanadu], [part], [vast]..."
1,The Rules of the Game,"[[Aviator], [André], [Jurieux], [Roland], [Tou..."
2,Bicycle Thieves,"[[postWorld], [War], [II], [Val], [Melaina], [..."
3,Vertigo (film),"[[rooftop], [chase], [fellow], [policeman], [f..."
4,The Godfather,"[[1945], [New], [York], [City], [daughter], [C..."


In [153]:
test_df.columns

Index(['Movie_Title', 'Plot After Stem'], dtype='object')

In [154]:
train_df.columns

Index(['Title', 'Genre Corrected', 'Plot After Stem'], dtype='object')

In [186]:
train_df['Plot After Stem len'] = train_df['Plot After Stem'].apply(len)
train_df.describe()

,Plot After Stem len
count,2500.000000
mean,193.513600
std,109.268455
min,5.000000
25%,98.000000
50%,206.000000
75%,268.000000
max,667.000000


In [179]:
train_df = train_df[train_df['Plot After Stem len'] > 0]

In [187]:
train_df.describe()

,Plot After Stem len
count,2500.000000
mean,193.513600
std,109.268455
min,5.000000
25%,98.000000
50%,206.000000
75%,268.000000
max,667.000000


In [180]:
train_df.head()

,Title,Genre Corrected,Plot After Stem,Plot After Stem len
0,Partners,Others,"[[series], [murders], [Los], [Angeless], [gay]...",93
1,Personal Best,Others,"[[Chris], [Cahill], [young], [athlete], [compe...",60
2,Poltergeist,Suspense,"[[Steven], [Diane], [Freeling], [live], [quiet...",250
3,Porky's,Others,"[[group], [Florida], [high], [school], [studen...",204
4,Q,Others,"[[Aztec], [god], [Quetzalcoatl], [winged], [dr...",121


In [184]:
test_df['Plot After Stem len'] = test_df['Plot After Stem'].apply(len)
test_df.describe()

,Plot After Stem len
count,998.000000
mean,234.524048
std,88.816722
min,1.000000
25%,187.000000
50%,248.500000
75%,292.000000
max,693.000000


In [185]:
test_df = test_df[test_df['Plot After Stem len'] > 0]
test_df.describe()

,Plot After Stem len
count,998.000000
mean,234.524048
std,88.816722
min,1.000000
25%,187.000000
50%,248.500000
75%,292.000000
max,693.000000


In [196]:
def lst_str (name):
    char = ' '
    for i in name:
        char = char + ' ' +str(i[0])
    return char

In [197]:
train_df['Plot'] = train_df['Plot After Stem'].apply(lst_str)

In [198]:
train_df.head()

,Title,Genre Corrected,Plot After Stem,Plot After Stem len,Plot
0,Partners,Others,"[[series], [murders], [Los], [Angeless], [gay]...",93,series murders Los Angeless gay community he...
1,Personal Best,Others,"[[Chris], [Cahill], [young], [athlete], [compe...",60,Chris Cahill young athlete competes unsucces...
2,Poltergeist,Suspense,"[[Steven], [Diane], [Freeling], [live], [quiet...",250,Steven Diane Freeling live quiet life Orange...
3,Porky's,Others,"[[group], [Florida], [high], [school], [studen...",204,group Florida high school students plan losi...
4,Q,Others,"[[Aztec], [god], [Quetzalcoatl], [winged], [dr...",121,Aztec god Quetzalcoatl winged dragonlike fem...


In [199]:
test_df['Plot'] = test_df['Plot After Stem'].apply(lst_str)
test_df.head()

,Movie_Title,Plot After Stem,Plot After Stem len,Plot
0,Citizen Kane,"[[mansion], [called], [Xanadu], [part], [vast]...",266,mansion called Xanadu part vast palatial est...
1,The Rules of the Game,"[[Aviator], [André], [Jurieux], [Roland], [Tou...",195,Aviator André Jurieux Roland Toutain lands L...
2,Bicycle Thieves,"[[postWorld], [War], [II], [Val], [Melaina], [...",186,postWorld War II Val Melaina neighbourhood R...
3,Vertigo (film),"[[rooftop], [chase], [fellow], [policeman], [f...",249,rooftop chase fellow policeman falls death S...
4,The Godfather,"[[1945], [New], [York], [City], [daughter], [C...",290,1945 New York City daughter Connies wedding ...


In [201]:
train_df.drop(['Plot After Stem','Plot After Stem len'], axis = 1, inplace = True)
train_df.head()

,Title,Genre Corrected,Plot
0,Partners,Others,series murders Los Angeless gay community he...
1,Personal Best,Others,Chris Cahill young athlete competes unsucces...
2,Poltergeist,Suspense,Steven Diane Freeling live quiet life Orange...
3,Porky's,Others,group Florida high school students plan losi...
4,Q,Others,Aztec god Quetzalcoatl winged dragonlike fem...


In [244]:
def str_lst (name):
    lst = []
    for i in name.split():
        lst.append(i)
    return lst

In [245]:
train_df['Genre Corrected'] = train_df['Genre Corrected'].apply(str_lst)
train_df.head()

,Title,Genre Corrected,Plot
0,Partners,[Others],series murders Los Angeless gay community he...
1,Personal Best,[Others],Chris Cahill young athlete competes unsucces...
2,Poltergeist,[Suspense],Steven Diane Freeling live quiet life Orange...
3,Porky's,[Others],group Florida high school students plan losi...
4,Q,[Others],Aztec god Quetzalcoatl winged dragonlike fem...


In [202]:
test_df.drop(['Plot After Stem','Plot After Stem len'], axis = 1, inplace = True)
test_df.head()

,Movie_Title,Plot
0,Citizen Kane,mansion called Xanadu part vast palatial est...
1,The Rules of the Game,Aviator André Jurieux Roland Toutain lands L...
2,Bicycle Thieves,postWorld War II Val Melaina neighbourhood R...
3,Vertigo (film),rooftop chase fellow policeman falls death S...
4,The Godfather,1945 New York City daughter Connies wedding ...


In [225]:
from sklearn.feature_extraction.text import CountVectorizer
# Might take awhile...
bow_transformer_tr = CountVectorizer(analyzer=text_process).fit(train_df['Plot'])
bow_transformer_te = CountVectorizer(analyzer=text_process).fit(test_df['Plot'])

# Print total number of vocab words
print(len(bow_transformer_tr.vocabulary_))
print(len(bow_transformer_tr.vocabulary_))

57640
57640


In [246]:
## for y_train

from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer_tr = MultiLabelBinarizer()
multilabel_binarizer_tr.fit(train_df['Genre Corrected'])
# transform target variable
y_train = multilabel_binarizer_tr.transform(train_df['Genre Corrected'])

In [247]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [248]:
x_tr, x_val, y_tr, y_val = train_test_split(train_df['Plot'],y_train, test_size=0.2, random_state=9)

In [249]:
x_tr_tfidf = tfidf_vectorizer.fit_transform(x_tr)
x_val_tfidf = tfidf_vectorizer.transform(x_val)

In [250]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [251]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [252]:
# fit model on train data
clf.fit(x_tr_tfidf, y_tr)

OneVsRestClassifier(estimator=LogisticRegression())

In [253]:
# make predictions for validation set
pred = clf.predict(x_val_tfidf)

In [254]:
pred[100]

array([0, 0, 1, 0, 0, 0])

In [255]:
multilabel_binarizer_tr.inverse_transform(pred)[100]

('Others',)

In [256]:
# evaluate performance
f1_score(y_val, pred, average="micro")

0.7228003784295176

In [259]:
# predict probabilities
pred_prob = clf.predict_proba(x_val_tfidf)
print(pred_prob)

[[0.10371595 0.02312847 0.82155902 0.03140976 0.02312847 0.07605453]
 [0.15138582 0.03628519 0.65764793 0.02327314 0.03628519 0.19845346]
 [0.13880578 0.02167131 0.78272774 0.02987664 0.02167131 0.09485555]
 ...
 [0.26031796 0.02249609 0.67031326 0.02583718 0.02249609 0.12810482]
 [0.09757133 0.03057063 0.84023717 0.03463912 0.03057063 0.09989641]
 [0.09210599 0.01743999 0.87767599 0.03048114 0.01743999 0.07867759]]


In [261]:
print(classification_report(y_val,pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        78
           1       0.00      0.00      0.00        13
           2       0.77      0.99      0.87       383
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        13
           5       1.00      0.03      0.06        60

   micro avg       0.77      0.68      0.72       563
   macro avg       0.30      0.17      0.16       563
weighted avg       0.63      0.68      0.60       563
 samples avg       0.76      0.72      0.73       563



c:\Miniconda\envs\py37_default\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Miniconda\envs\py37_default\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [307]:
test_df['Plot len'] = test_df['Plot'].apply(len)
test_df.head(20)

,Movie_Title,Plot,Transformed Plot Matrix,Predicted Genre,Plot len
0,Citizen Kane,mansion called Xanadu part vast palatial est...,0,"[(Others,), (), (Suspense,)]",2026
1,The Rules of the Game,Aviator André Jurieux Roland Toutain lands L...,0,"[(Others,), (), (Suspense,)]",1428
2,Bicycle Thieves,postWorld War II Val Melaina neighbourhood R...,0,"[(Others,), (), (Suspense,)]",1353
3,Vertigo (film),rooftop chase fellow policeman falls death S...,0,"[(Others,), (), (Suspense,)]",1882
4,The Godfather,1945 New York City daughter Connies wedding ...,0,"[(Others,), (), (Suspense,)]",2162
5,Seven Samurai,nan,0,"[(Others,), (), (Suspense,)]",5
6,Psycho (1960 film),lunchtime tryst Phoenix Arizona hotel reales...,0,"[(Others,), (), (Suspense,)]",2140
7,Singin' in the Rain,Lockwood Gene Kelly popular silent film star...,0,"[(Others,), (), (Suspense,)]",2362
8,The Searchers,1868 Ethan Edwards returns eightyear absence...,0,"[(Others,), (), (Suspense,)]",2099
9,8½,Guido Anselmi Marcello Mastroianni famous It...,0,"[(Others,), (), (Suspense,)]",2251


In [308]:
test_df = test_df[test_df['Plot len'] > 10]

In [348]:
x_te_tfidf = tfidf_vectorizer.fit_transform(test_df['Plot'])

In [349]:
pred_test = clf.predict(x_te_tfidf)

In [350]:
pred_test[100]

array([0, 0, 1, 0, 0, 0])

In [351]:
multilabel_binarizer_tr.inverse_transform(pred_test)[100]

('Others',)

In [367]:
test_df.head()

,Movie_Title,Plot,Transformed Plot Matrix,Predicted Genre,Plot len
0,Citizen Kane,mansion called Xanadu part vast palatial est...,0,"[(Others,), (), (Suspense,)]",2026
1,The Rules of the Game,Aviator André Jurieux Roland Toutain lands L...,0,"[(Others,), (), (Suspense,)]",1428
2,Bicycle Thieves,postWorld War II Val Melaina neighbourhood R...,0,"[(Others,), (), (Suspense,)]",1353
3,Vertigo (film),rooftop chase fellow policeman falls death S...,0,"[(Others,), (), (Suspense,)]",1882
4,The Godfather,1945 New York City daughter Connies wedding ...,0,"[(Others,), (), (Suspense,)]",2162


In [393]:
def infer_tags(q):
    q = text_process(q)
    q = stem(q)
    q_str = lst_str(q)
    q_str = str(q_str)
    print(q_str)
    q_vec = tfidf_vectorizer.fit_transform(q_str)
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

In [394]:
test_df['Predicted Genre'] = test_df['Plot'].apply(infer_tags)
test_df.head()

  mansion called Xanadu part vast palatial estate Florida elderly Charles Foster Kane deathbed Holding snow globe utters word Rosebud dies slips hand smashes floor newsreel obituary tells life story enormously wealthy newspaper publisher industrial magnate Kanes death becomes sensational news around world newsreels producer tasks reporter Jerry Thompson discovering meaning sets interview friends associates tries approach wife Susan Alexander alcoholic runs nightclub refuses talk goes private archive late banker Walter Parks Thatcher Thatchers written memoirs learns rise decline personal fortune 1871 gold discovered mining deed belonging mother Mary hired establish trust would provide education assume guardianship parents discussed arrangements inside young played happily sled outside boardinghouse turned boy struck attempted run away time gained control age 25 mines productivity prudent investing made one richest men took New York Inquirer embarked career yellow journalism publishing s

ValueError: Iterable over raw text documents expected, string object received.

In [382]:
lst_demo = ['a','b','c']
rs = ' '.join(lst_demo)
print(rs)
print(type(rs))

a b c
<class 'str'>


In [378]:
numList = ['1', '2', '3', '4']
separator = ', '
print(separator.join(numList))

1, 2, 3, 4


In [299]:
test_df.describe()

,Movie_Title,Plot,Transformed Plot Matrix,Predicted Genre
count,998,998,998,998
unique,998,983,1,1
top,Pinocchio (1940 film),nan,"(0, 9984)\t0.034491209491660645\n (0, 9980)...","[(Others,), (), (Suspense,)]"
freq,1,16,998,998


## SVM scores

In [215]:
predictions = model.predict(x_val_tfidf)
print(confusion_matrix(y_val,predictions))

[[  2   0   0  53   0   0   0   0   0   0]
 [  0   0   0  22   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   0   0   0]
 [  0   0   0 334   0   0   0   0   0   0]
 [  0   0   0  12   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   0   0   0]
 [  0   0   0  14   0   0   0   0   0   0]
 [  0   0   0   4   0   0   0   0   0   0]
 [  0   0   0  12   0   0   0   0   0   0]
 [  0   0   0  43   0   0   0   0   0   2]]


In [217]:
print(classification_report(y_val,predictions))

                         precision    recall  f1-score   support

                 Action       1.00      0.04      0.07        55
          Action Others       0.00      0.00      0.00        22
        Action Suspense       0.00      0.00      0.00         1
                 Others       0.67      1.00      0.80       334
         Others Romance       0.00      0.00      0.00        12
 Others Science Fiction       0.00      0.00      0.00         1
        Others Suspense       0.00      0.00      0.00        14
                Romance       0.00      0.00      0.00         4
        Science Fiction       0.00      0.00      0.00        12
               Suspense       1.00      0.04      0.09        45

               accuracy                           0.68       500
              macro avg       0.27      0.11      0.10       500
           weighted avg       0.65      0.68      0.55       500



c:\Miniconda\envs\py37_default\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [218]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 

In [ ]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
grid.fit(X_train,y_train)